<a href="https://colab.research.google.com/github/NathanyApSalles/analysis_foodtech/blob/main/Case_Tecnico_DataAnalyst_Ifood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importação das bases

In [46]:
import requests
import tarfile
import os
from pyspark.sql import SparkSession, DataFrame

spark = SparkSession.builder.getOrCreate()

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import shapiro, anderson, boxcox, mannwhitneyu
import numpy as np
import seaborn as sns

In [ ]:
def read_file(url: str, local_path: str, type_file: str) -> DataFrame:
  """Função para ler arquivo e retornar um Dataframe."""
  if not os.path.exists(local_path):
    response = requests.get(url)
    with open(local_path, "wb") as f:
        f.write(response.content)
  if type_file == "json":
    return spark.read.json(local_path, multiLine=False)
  elif type_file == "csv":
    return spark.read.option("header", "true").csv(local_path)
  elif type_file == "tar":

    arquivos_extraidos = "/tmp/ab_test_ref"

    os.makedirs(arquivos_extraidos, exist_ok=True)

    with tarfile.open(local_path, "r:gz") as tar:
        tar.extractall(path=arquivos_extraidos)

    filename = ""
    for root, dirs, files in os.walk(arquivos_extraidos):
        for filename in files:
            print(filename) # print para visualizar todos os arquivos exraídos
    if ".csv" in filename: # se houver algum arquivo csv, junte todos os arquivos deste tipo no dataframe
      return spark.read.option("header", "true").csv(arquivos_extraidos + "/*.csv")
    else: # se houver algum arquivo json, junte todos os arquivos deste tipo no dataframe
      return spark.read.json(arquivos_extraidos + "/*.json")
      #pode acontecer de ter arquivos de diferentes tipos misturados, mas para este estudo vamos assumir que todos são do mesmo tipo


In [ ]:
url_pedidos = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/order.json.gz"
url_usuarios = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/consumer.csv.gz"
url_merchants = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/restaurant.csv.gz"
url_test_ab = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/ab_test_ref.tar.gz"

local_path_pedidos = "/tmp/order.json.gz"
local_path_usuarios = "/tmp/consumer.csv.gz"
local_path_merchants = "/tmp/restaurant.csv.gz"
local_path_teste_ab = "/tmp/ab_test_ref.tar.gz"

pedidos = read_file(url_pedidos, local_path_pedidos, "json").cache()
usuarios = read_file(url_usuarios, local_path_usuarios, "csv").cache()
merchants = read_file(url_merchants, local_path_merchants, "csv").cache()
teste_ab = read_file(url_test_ab, local_path_teste_ab, "tar").cache()

In [ ]:
pedidos.show(5)

In [ ]:
usuarios.show(5)

In [ ]:
merchants.show(5)

In [ ]:
teste_ab.show(5, truncate=False)

In [ ]:
from pyspark.sql.functions import col, broadcast, when, count, isnan, count_distinct, sum, avg, row_number, lag, date_diff, round, max, to_date, lit
from pyspark.sql.window import Window

# Entendendo os dados

In [ ]:
def df_info(df: DataFrame, colunas: list) -> None:
  """Função para auxiliar a identificar tamanho da base, tipo das colunas,
   valores nulos, duplicidade em colunas específicas na base."""
  # validando o tipo das colunas
  df.printSchema()

  # validando o tamanho da base
  print(f"Qtd de linhas: {df.count()}")

  print("\nValores nulos")
  df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

  print("\nValidando valores duplicados")
  for column in colunas:
    num_duplicados = df.groupBy(column).count().where(col("count") > 1).count()
    print(f"\n{column}: {num_duplicados}")

In [ ]:
df_info(usuarios, ["customer_id"])

In [47]:
df_info(pedidos, ["order_id"])

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
# removendo os usuários nulos da base
# na base de pedidos temos um número pequeno de pedidos sem customer_id atribuído,
# porém como o objetivo da análise é validar os resultados do teste a/b, estes pedidos serão removidos visto que
# não participaram do teste a/b
pedidos_validos_ab = pedidos.where(col("customer_id").isNotNull())

In [ ]:
# podemos ver que a base de pedidos possui uma grande voliumentria de pedidos duplicados
# foi analisado uma amostra e notou-se que para os pedidos duplicados, os campos que diferente são CPF e data de criação do pedido
# pode ter acontecido algum problema no produto ao gerar o número do pedido, ou até mesmo ao gerar a base,
# diantes disto podemos seguir com algumas tratativas, como criar um novo order_id concatenando com o CPF, ou
# dentro dos duplicados manter o pedido mais atual, ou a mais antigo.
# Como os valores dos pedidos são iguais, merchants, itens também são idênticos, optou-se por manter o pedido mais antigo


pedidos_validos_ab = (pedidos_validos_ab
                      .withColumn("rank", row_number().over(Window.partitionBy("order_id").orderBy("order_created_at")))
                      .where(col("rank") == 1)
                      .drop("rank")
).cache()
df_info(pedidos_validos_ab, ["order_id"])

In [ ]:
df_info(merchants, ["id"])

In [ ]:
df_info(teste_ab, ["customer_id"])

# EDA

In [ ]:
# tem no teste_Ab mas não comprou? ou seja, não está na base de pedidos?
c = pedidos_validos_ab.select(col("customer_id").alias("custumer_with_order")).distinct()
teste_ab.join(c, col("customer_id") == col("custumer_with_order"), "left").where(col("custumer_with_order").isNull()).show()

# Sugestão de Teste A/B
- Todos os usuários do teste a/b tem algum pedido, logo, uma sugestão é ter uma base sobre todas os usuários que receberam cupons para conseguirmos calcular conversão;
- Teste com diferentes valores de cupons para entender se o valor do desconto impacta no ticket médio;
- Frete grátis: entender se cupons de frete grátis aumentam o ticket médio;
- Avaliar base de de satisfação (NPS) para entender se a disponibilização de cupons impactam na experiencia e satisfação do usuário;
- Carrinho abandonado: teste a/b de envio de cupons para usuários que abandonaram carrinho e analisar a finalização da compra (conversão). Aplicável principalmente para usuários de produtos de farmácia e mercado, visto que a compra de refeições é feita por usuários que já esperam uma entrega rápida.

In [ ]:
teste_ab.groupBy(col("is_target")).agg(count_distinct("customer_id")).show()

In [ ]:
pedidos_final = pedidos_validos_ab.join(broadcast(teste_ab), ['customer_id'], "left").cache()
#pedidos_final.show(5)

In [ ]:
#pedidos_final.where(col("is_target").isNull()).show()

In [ ]:
pedidos_final.groupBy(col("is_target")).agg(
                                      count_distinct(col("order_id")).alias("distinct_orders"),
                                      count_distinct(col("customer_id")).alias("distinct_customers"),
                                      count_distinct(col("merchant_id")).alias("distinct_merchants"),
                                      sum(col("order_total_amount")).alias("order_total_amount"),
                                      avg(col("order_total_amount")).alias("order_avg_amount"),
                                      (sum(col("order_total_amount"))/count_distinct(col("order_id"))).alias("ticket_medio"),


                                      ).show()

# tempo entre compras

In [ ]:
df = pedidos_final.withColumn("data_criacao_pedido", col("order_created_at").cast("date")).select("customer_id", "is_target", "data_criacao_pedido")

df_tempo_entre_compras = df.withColumn("data_pedido_anterior", lag("data_criacao_pedido") \
                                       .over(Window.partitionBy("customer_id").orderBy("data_criacao_pedido"))) \
                            .withColumn("tempo_entre_compras", date_diff("data_criacao_pedido", "data_pedido_anterior"))

df_tempo_entre_compras.show()

In [ ]:
tempo_medio_entre_compras = df_tempo_entre_compras.where(col("tempo_entre_compras").isNotNull()).groupBy("is_target").agg(avg("tempo_entre_compras"))
tempo_medio_entre_compras.show()

In [ ]:
tempo_medio_entre_compras = df_tempo_entre_compras.groupBy("customer_id","is_target") \
.agg(round(avg("tempo_entre_compras"),2).alias("tempo_medio_entre_compras"),
     max("data_criacao_pedido").alias("data_ultima_compra")) \
.withColumn("tempo_medio_entre_compras", when(col("tempo_medio_entre_compras").isNull(), 0).otherwise(col("tempo_medio_entre_compras"))) \
.withColumn("recencia", date_diff(to_date(lit("2019-02-01")), "data_ultima_compra")) \
.drop(col("data_ultima_compra"))


tempo_medio_entre_compras.show()

In [ ]:
# qtd diferente de merchants, diversificação de estabelecimentos, entender se os clientes tem algum estabelecimento onde mais compram,
# ou se compram em vários

In [ ]:
frequencia_valor = pedidos_final.groupBy(col("customer_id"),col("is_target")) \
.agg(
      count_distinct(col("order_id")).alias("distinct_orders"),
      count_distinct(col("merchant_id")).alias("distinct_merchants"),
      round(sum(col("order_total_amount")),2).alias("order_total_amount"),
      round(avg(col("order_total_amount")),2).alias("order_avg_amount"),
      round((sum(col("order_total_amount"))/count_distinct(col("order_id"))),2).alias("ticket_medio"))
frequencia_valor.show()

In [44]:
perfil_compra = tempo_medio_entre_compras.alias("t") \
.join(frequencia_valor.alias("f"), ["customer_id"], "inner") \
.withColumn("is_target_dummy", when(col("t.is_target") == "target", lit("1")).otherwise(lit("0"))) \
.select("t.customer_id",
        "is_target_dummy",
        "tempo_medio_entre_compras",
        "recencia",
        "distinct_orders",
        "distinct_merchants",
        "order_total_amount",
        "order_avg_amount",
        "ticket_medio"
        )

perfil_compra.show()

ConnectionRefusedError: [Errno 111] Connection refused

In [37]:
pd_perfil_compra = perfil_compra.toPandas()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [35]:
#correlação de pearson
plt.figure(figsize=(20,8))
sns.heatmap(pd_perfil_compra.corr(), annot = True, cmap = "RdYlGn");
plt.title('Correlação de Pearson', size = 15)

NameError: name 'pd_perfil_compra' is not defined

<Figure size 2000x800 with 0 Axes>



---


# Teste de Hipótese:

In [ ]:
#qtd de pedidos por usuário
df_media_pedidos = pedidos_final.groupBy(col("customer_id"), col("is_target")).agg(count_distinct("order_id").alias("qtd_pedidos_por_usuario"))

df_media_pedidos.groupBy(col("is_target")).agg(avg(col("qtd_pedidos_por_usuario")).alias("media_pedidos_por_usuario")).show()

# Usuários target compram mais do que usuários control?
Primeiramente vamos validar se os dados possuem uma distribuição normal

##Teste de Shapiro Wilk

**- H0:** A amostra segue uma distribuição normal

**- H1:** A amostra não segue uma distribuição normal
- Se _p-value_ < 0.05, rejeita-se a hipótese nula, ou seja, temos evidências suficientes para dizer que a amostra não vem de distribuição normal


In [ ]:
pd_pedidos = df_media_pedidos.toPandas()

In [ ]:
# analisando se os dados obedecem uma distribuição normal
statistic, pvalue = shapiro(pd_pedidos["qtd_pedidos_por_usuario"])
print(f"statistic={statistic}, pvalue={pvalue}")
if pvalue < 0.05:
    print('Rejeita a hipótese nula')
else:
    print('Não rejeita a hipótese nula')

Como os dados não seguem uma distribuição normal, logo, seguiremos por testes não paramétricos

## Hipóteses
**H0:** quantidade de pedidos do grupo target é igual ao grupo control

**H1:** quantidade de pedidos do grupo target é diferente ao grupo control

In [ ]:
target = pd_pedidos[pd_pedidos["is_target"]=="target"]
control = pd_pedidos[pd_pedidos["is_target"]=="control"]

statistic, pvalue = mannwhitneyu(target["qtd_pedidos_por_usuario"], control["qtd_pedidos_por_usuario"])
print(f"statistic={statistic}, pvalue={pvalue}")

if pvalue < 0.05:
    print('Rejeita a hipótese nula')
else:
    print('Não rejeita a hipótese nula')

#tamanho de efeito

print(statistic/(len(target["qtd_pedidos_por_usuario"])*len(control["qtd_pedidos_por_usuario"])))

## Resultado:
Rejeitamos a hipótese nula (H₀) e aceitamos a hipótese alternativa, ou seja, há uma diferença significativa entre a quantidade de pedidos dos dois grupos. O tamanho de efeito nos informa a magnitude desta diferença, sendo um índice entre 0 e 1. O tamanho do efeito calculado é 0.55, o que sugere um efeito de tamanho moderado. Isso significa que a diferença na quantidade de pedidos é significativa além de ser relevante.


---


## Vamos analisar agora se a diferença é para mais ou para menos
**H0:** usuários do grupo target tem igual ou menor quatidade de pedidos que o contro control

**H1:** usuários do grupo target tem maior quatidade de pedidos que o contro control

In [ ]:

target = pd_pedidos[pd_pedidos["is_target"]=="target"]
control = pd_pedidos[pd_pedidos["is_target"]=="control"]

statistic, pvalue = mannwhitneyu(target["qtd_pedidos_por_usuario"], control["qtd_pedidos_por_usuario"], alternative='greater')
print(f"statistic={statistic}, pvalue={pvalue}")

if pvalue < 0.05:
    print('Rejeita a hipótese nula')
else:
    print('Não rejeita a hipótese nula')

#tamanho de efeito

print(statistic/(len(target["qtd_pedidos_por_usuario"])*len(control["qtd_pedidos_por_usuario"])))

## Resultado do teste de hipótese:
Novamente rejeitamos a hipótese nula (H₀) concluindo que o grupo target realizou mais pedidos em relação ao grupo control
O tamanho de efeito nos informa a magnitude desta diferença, sendo um índice entre 0 e 1. O tamanho do efeito calculado é 0.55, o que sugere um efeito de tamanho moderado. Isso significa que a diferença no tempo de atendimento é significativa além de ser relevante.



---




- tarquet x control

    - qtd de pedidos
    - valor total da compra
    - ticket médio
    - tempo entre compras
    - recorrencia
    - tem diferença entre usuários ativos ou não?
    - expansão geográfica: qtd de estabelecimentos diferentes que compraram
    - restaurantes que mais venderam
    - região que mais vendeu
    - horário das compras
    - qtd de produtos adquiridos

- ideias para teste ab:

    - qtd de usuários que receberam o cupom, para entender conversão, de quem recebeu, quem não comprou;
     